In [15]:
! pip install -q -U torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 accelerate trl==0.4.7 tqdm python-dotenv

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      /private/var/folders/0s/xq09hn4d59v75lhpt4wwsm980000gn/T/pip-build-env-ejnetp91/overlay/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: Apache Software License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_p

In [2]:
import os
from dataclasses import dataclass, field
from typing import Optional
from datasets import load_dataset, Dataset
import torch
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    HfArgumentParser,
)
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

/Users/psykick/Documents/GitHub/fine-tuning-qlora/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
login(token=os.environ.get("HUGGINGFACE_TOKEN"))

In [4]:
dataset = load_dataset("gbharti/finance-alpaca")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 68912
    })
})


In [5]:
n = 100
text_data = []
for point in dataset["train"]:
    text = f"[INST] {point['instruction']} [/INST] {point['output']}"
    text_data.append({'text': text})
text_data = text_data[:n]

text_dataset = Dataset.from_list(text_data)
del(dataset)
print(text_dataset)

Dataset({
    features: ['text'],
    num_rows: 100
})


In [6]:
BASE_MODEL = "google/gemma-3-1b-pt"
NEW_MODEL = "finance-gemma"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.padding_side = "right"

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    trust_remote_code=True,
    flash_attention_2=True,
    flash_rotary = True,
    fused_dense = True,
    low_cpu_mem_usage=True,
    device_map="auto",
    revision="main",
)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [12]:
! pip install bitsandbytes==0.40.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 12.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0
